In [8]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflowlite (from versions: none)
ERROR: No matching distribution found for tensorflowlite


In [3]:
pip install supervision

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
from collections import defaultdict
import time

class ObjectDetectionTracker:
    def __init__(self, model_path='yolov8n.pt', confidence_threshold=0.5, iou_threshold=0.3):
        """
        Initialize the object detection and tracking system
        """
        self.model = YOLO(model_path)
        self.conf_threshold = confidence_threshold
        self.iou_threshold = iou_threshold
        self.tracking_objects = {}
        self.track_id = 0
        self.fps_history = []
        
    def calculate_iou(self, box1, box2):
        """Calculate Intersection over Union between two boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0

    def process_frame(self, frame):
        """
        Process a single frame for object detection and tracking
        """
        # Store original frame dimensions
        height, width = frame.shape[:2]
        
        # Detect objects
        results = self.model(frame, stream=True)
        
        # Process detection results
        current_objects = {}
        
        for result in results:
            boxes = result.boxes.cpu().numpy()
            
            for box in boxes:
                # Get detection information
                x1, y1, x2, y2 = box.xyxy[0].astype(int)
                conf = box.conf[0]
                class_id = int(box.cls[0])
                class_name = self.model.names[class_id]
                
                if conf < self.conf_threshold:
                    continue
                    
                # Check if object overlaps with any existing tracked object
                matched = False
                for track_id, tracked_obj in self.tracking_objects.items():
                    tracked_box = tracked_obj['box']
                    iou = self.calculate_iou([x1, y1, x2, y2], tracked_box)
                    
                    if iou > self.iou_threshold:
                        current_objects[track_id] = {
                            'box': [x1, y1, x2, y2],
                            'class': class_name,
                            'confidence': conf,
                            'tracked_frames': tracked_obj['tracked_frames'] + 1
                        }
                        matched = True
                        break
                
                # If no match found, create new tracked object
                if not matched:
                    self.track_id += 1
                    current_objects[self.track_id] = {
                        'box': [x1, y1, x2, y2],
                        'class': class_name,
                        'confidence': conf,
                        'tracked_frames': 1
                    }
        
        # Update tracking objects
        self.tracking_objects = current_objects
        
        return self.tracking_objects

    def draw_results(self, frame, objects):
        """
        Draw detection and tracking results on frame
        """
        for track_id, obj in objects.items():
            x1, y1, x2, y2 = obj['box']
            class_name = obj['class']
            conf = obj['confidence']
            tracked_frames = obj['tracked_frames']
            
            # Draw box with different colors based on tracking duration
            color = (0, int(min(255, tracked_frames * 10)), 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Draw label with tracking ID
            label = f"#{track_id} {class_name} {conf:.2f}"
            cv2.putText(frame, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
        return frame

    def run_detection(self, source=0, display=True):
        """
        Run real-time detection and tracking
        """
        cap = cv2.VideoCapture(source)
        
        if not cap.isOpened():
            print("Error: Couldn't open video source")
            return
        
        print("Detection and tracking started... Press 'q' to quit")
        
        try:
            while True:
                start_time = time.time()
                
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Process frame
                tracked_objects = self.process_frame(frame)
                
                # Draw results
                if display:
                    frame = self.draw_results(frame, tracked_objects)
                    
                    # Calculate and display FPS
                    fps = 1.0 / (time.time() - start_time)
                    self.fps_history.append(fps)
                    if len(self.fps_history) > 30:
                        self.fps_history.pop(0)
                    avg_fps = sum(self.fps_history) / len(self.fps_history)
                    
                    cv2.putText(frame, f'FPS: {avg_fps:.1f}', (10, 30),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
                    cv2.imshow('Object Detection and Tracking', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            cap.release()
            if display:
                cv2.destroyAllWindows()

def main():
    # Initialize detector with custom settings
    detector = ObjectDetectionTracker(
        model_path='yolov8n.pt',
        confidence_threshold=0.5,
        iou_threshold=0.3
    )
    
    # Run detection
    detector.run_detection()

if __name__ == "__main__":
    main()

Detection and tracking started... Press 'q' to quit

0: 480x640 2 persons, 1 remote, 498.3ms
Speed: 22.8ms preprocess, 498.3ms inference, 27.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 tie, 194.6ms
Speed: 9.7ms preprocess, 194.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 2 ties, 188.0ms
Speed: 9.2ms preprocess, 188.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 tie, 200.0ms
Speed: 4.0ms preprocess, 200.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 tie, 218.0ms
Speed: 5.0ms preprocess, 218.0ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 tie, 235.8ms
Speed: 6.0ms preprocess, 235.8ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 tie, 273.0ms
Speed: 4.0ms preprocess, 273.0ms inference, 2.0ms postprocess per image at shape (1, 3, 

In [2]:
#quantized model

#i downloaded the quantized version  by quantizing
# size reduced to half s

import cv2
from ultralytics import YOLO
import numpy as np
from collections import defaultdict
import time
import torch

class ObjectDetectionTracker:
    def __init__(self, model_path='yolov8n_quantized.pt', confidence_threshold=0.5, iou_threshold=0.3):
        """
        Initialize the object detection and tracking system with quantized model
        """
        try:
            # First try loading directly
            self.model = YOLO(model_path)
        except KeyError:
            # If that fails, try loading as a state dict
            print("Attempting to load quantized model as state dict...")
            base_model = YOLO('yolov8n.pt')  # Load base model first
            state_dict = torch.load(model_path)
            
            # Handle different possible state dict formats
            if isinstance(state_dict, dict):
                if 'state_dict' in state_dict:
                    state_dict = state_dict['state_dict']
                elif 'model' in state_dict:
                    state_dict = state_dict['model']
            
            base_model.model.load_state_dict(state_dict, strict=False)
            self.model = base_model
            
        self.conf_threshold = confidence_threshold
        self.iou_threshold = iou_threshold
        self.tracking_objects = {}
        self.track_id = 0
        self.fps_history = []
        
    def calculate_iou(self, box1, box2):
        """Calculate Intersection over Union between two boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0

    def process_frame(self, frame):
        """
        Process a single frame for object detection and tracking
        """
        # Store original frame dimensions
        height, width = frame.shape[:2]
        
        # Detect objects
        results = self.model(frame, stream=True)
        
        # Process detection results
        current_objects = {}
        
        for result in results:
            boxes = result.boxes.cpu().numpy()
            
            for box in boxes:
                # Get detection information
                x1, y1, x2, y2 = box.xyxy[0].astype(int)
                conf = box.conf[0]
                class_id = int(box.cls[0])
                class_name = self.model.names[class_id]
                
                if conf < self.conf_threshold:
                    continue
                    
                # Check if object overlaps with any existing tracked object
                matched = False
                for track_id, tracked_obj in self.tracking_objects.items():
                    tracked_box = tracked_obj['box']
                    iou = self.calculate_iou([x1, y1, x2, y2], tracked_box)
                    
                    if iou > self.iou_threshold:
                        current_objects[track_id] = {
                            'box': [x1, y1, x2, y2],
                            'class': class_name,
                            'confidence': conf,
                            'tracked_frames': tracked_obj['tracked_frames'] + 1
                        }
                        matched = True
                        break
                
                # If no match found, create new tracked object
                if not matched:
                    self.track_id += 1
                    current_objects[self.track_id] = {
                        'box': [x1, y1, x2, y2],
                        'class': class_name,
                        'confidence': conf,
                        'tracked_frames': 1
                    }
        
        # Update tracking objects
        self.tracking_objects = current_objects
        
        return self.tracking_objects

    def draw_results(self, frame, objects):
        """
        Draw detection and tracking results on frame
        """
        for track_id, obj in objects.items():
            x1, y1, x2, y2 = obj['box']
            class_name = obj['class']
            conf = obj['confidence']
            tracked_frames = obj['tracked_frames']
            
            # Draw box with different colors based on tracking duration
            color = (0, int(min(255, tracked_frames * 10)), 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Draw label with tracking ID
            label = f"#{track_id} {class_name} {conf:.2f}"
            cv2.putText(frame, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
        return frame

    def run_detection(self, source=0, display=True):
        """
        Run real-time detection and tracking
        """
        cap = cv2.VideoCapture(source)
        
        if not cap.isOpened():
            print("Error: Couldn't open video source")
            return
        
        print("Detection and tracking started... Press 'q' to quit")
        
        try:
            while True:
                start_time = time.time()
                
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Process frame
                tracked_objects = self.process_frame(frame)
                
                # Draw results
                if display:
                    frame = self.draw_results(frame, tracked_objects)
                    
                    # Calculate and display FPS
                    fps = 1.0 / (time.time() - start_time)
                    self.fps_history.append(fps)
                    if len(self.fps_history) > 30:
                        self.fps_history.pop(0)
                    avg_fps = sum(self.fps_history) / len(self.fps_history)
                    
                    cv2.putText(frame, f'FPS: {avg_fps:.1f}', (10, 30),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
                    cv2.imshow('Object Detection and Tracking', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            cap.release()
            if display:
                cv2.destroyAllWindows()

def main():
    # Initialize detector with quantized model
    detector = ObjectDetectionTracker(
        model_path='yolov8n_quantized.pt',  # Using the quantized model
        confidence_threshold=0.5,
        iou_threshold=0.3
    )
    
    # Run detection
    detector.run_detection()

if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'yolov8n_quantized.pt'

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
from pathlib import Path
import matplotlib.pyplot as plt
import torch

class YOLOModelComparator:
    def __init__(self):
        """
        Initialize with default YOLO models
        """
        # Dictionary to store model names and their corresponding YOLO models
        self.models = {
            'YOLOv8n': YOLO('yolov8n.pt'),
            'YOLOv8s': YOLO('yolov8s.pt')
        }
        
        # COCO class names for label mapping
        self.class_names = [
            'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck',
            'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
            'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
            'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
            'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
            'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
            'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
            'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
            'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse',
            'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
            'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
            'toothbrush'
        ]
        
    def detect_objects(self, image_path, conf_threshold=0.25):
        """
        Perform object detection using all models
        """
        results = {}
        
        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image at {image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Run detection with each model
        for model_name, model in self.models.items():
            detections = model(image, conf=conf_threshold)[0]
            
            # Extract detection information
            boxes = detections.boxes.xyxy.cpu().numpy()
            confidences = detections.boxes.conf.cpu().numpy()
            class_ids = detections.boxes.cls.cpu().numpy().astype(int)
            
            # Store results
            results[model_name] = {
                'boxes': boxes,
                'confidences': confidences,
                'class_ids': class_ids,
                'labels': [self.class_names[id] for id in class_ids]
            }
            
        return results, image

    def visualize_detections(self, image_path, conf_threshold=0.25):
        """
        Visualize detections from all models side by side
        """
        # Get detections
        try:
            results, image = self.detect_objects(image_path, conf_threshold)
        except Exception as e:
            print(f"Error during detection: {e}")
            return None

        # Create figure
        num_models = len(self.models)
        fig, axes = plt.subplots(1, num_models, figsize=(15, 5))
        if num_models == 1:
            axes = [axes]

        # Color map for different classes
        colors = plt.cm.hsv(np.linspace(0, 1, 80)).tolist()

        # Plot detections for each model
        for ax, (model_name, detections) in zip(axes, results.items()):
            ax.imshow(image)
            
            # Draw boxes and labels for each detection
            for box, conf, label, class_id in zip(
                detections['boxes'],
                detections['confidences'],
                detections['labels'],
                detections['class_ids']
            ):
                x1, y1, x2, y2 = box.astype(int)
                color = colors[class_id]
                
                # Draw box
                rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                   fill=False,
                                   color=color,
                                   linewidth=2)
                ax.add_patch(rect)
                
                # Add label
                label_text = f'{label} {conf:.2f}'
                ax.text(x1, y1-10, label_text,
                       color=color,
                       fontsize=8,
                       bbox=dict(facecolor='white', alpha=0.7))

            ax.set_title(f'{model_name}\nDetections: {len(detections["boxes"])}')
            ax.axis('off')

        plt.tight_layout()
        return fig

    def compare_performance(self, image_path, num_runs=5):
        """
        Compare model performance metrics
        """
        performance_metrics = {}
        
        for model_name, model in self.models.items():
            # Measure inference time
            times = []
            for _ in range(num_runs):
                start_time = time.time()
                _ = model(image_path)
                times.append(time.time() - start_time)
            
            # Get detections for object count
            results = model(image_path)[0]
            num_objects = len(results.boxes)
            
            performance_metrics[model_name] = {
                'avg_inference_time': np.mean(times),
                'std_inference_time': np.std(times),
                'num_objects_detected': num_objects
            }
            
        return performance_metrics

def main():
    # Create comparator instance
    comparator = YOLOModelComparator()
    
    # Path to your test image
    image_path = 'images/3892.jpg'  # Replace with your image path
    
    try:
        # Visualize detections
        fig = comparator.visualize_detections(image_path, conf_threshold=0.25)
        if fig:
            plt.show()
        
        # Compare performance
        metrics = comparator.compare_performance(image_path)
        
        # Print performance metrics
        print("\nPerformance Metrics:")
        for model_name, metric in metrics.items():
            print(f"\n{model_name}:")
            print(f"Average Inference Time: {metric['avg_inference_time']:.3f} seconds")
            print(f"Standard Deviation: {metric['std_inference_time']:.3f} seconds")
            print(f"Objects Detected: {metric['num_objects_detected']}")
            
    except Exception as e:
        print(f"Error during comparison: {e}")

if __name__ == "__main__":
    main()

In [13]:
#we can clearlyy see yolov8s  >> yolov8n
#now comparre with quantized one

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import torch

class QuantizedDetector:
    def __init__(self, model_path):
        """
        Initialize with quantized model path
        """
        try:
            # Load the base model first
            self.model = YOLO('yolov8n.pt')
            
            # Load the quantized state dict
            state_dict = torch.load(model_path)
            if 'state_dict' in state_dict:
                self.model.model.load_state_dict(state_dict['state_dict'])
            elif 'model' in state_dict:
                self.model.model.load_state_dict(state_dict['model'])
            else:
                self.model.model.load_state_dict(state_dict)
                
            print(f"Model loaded successfully from {model_path}")
            
            # Move model to GPU if available
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            self.model.to(self.device)
            
        except Exception as e:
            print(f"Error loading model: {e}")
            raise

        # COCO class names
        self.class_names = [
            'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck',
            'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
            'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
            'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
            'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
            'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
            'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
            'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
            'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse',
            'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
            'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
            'toothbrush'
        ]

    def detect_objects(self, image_path, conf_threshold=0.25):
        """
        Detect objects in an image and return detailed results
        """
        try:
            # Read image
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Could not read image at {image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Time the inference
            start_time = time.time()
            results = self.model(image, conf=conf_threshold)[0]
            inference_time = time.time() - start_time

            # Extract detection information
            boxes = results.boxes.xyxy.cpu().numpy()
            confidences = results.boxes.conf.cpu().numpy()
            class_ids = results.boxes.cls.cpu().numpy().astype(int)

            # Count objects by class
            class_counts = {}
            for class_id, conf in zip(class_ids, confidences):
                class_name = self.class_names[class_id]
                if class_name not in class_counts:
                    class_counts[class_name] = {
                        'count': 0,
                        'confidences': []
                    }
                class_counts[class_name]['count'] += 1
                class_counts[class_name]['confidences'].append(conf)

            detection_results = {
                'boxes': boxes,
                'confidences': confidences,
                'class_ids': class_ids,
                'class_counts': class_counts,
                'total_objects': len(boxes),
                'inference_time': inference_time
            }

            return detection_results, image

        except Exception as e:
            print(f"Error during detection: {e}")
            raise

    def visualize_detections(self, image_path, conf_threshold=0.25):
        """
        Visualize detected objects with bounding boxes and labels
        """
        try:
            # Get detections
            results, image = self.detect_objects(image_path, conf_threshold)

            # Create figure
            plt.figure(figsize=(12, 8))
            plt.imshow(image)

            # Color map for different classes
            colors = plt.cm.hsv(np.linspace(0, 1, 80)).tolist()

            # Draw boxes and labels
            for box, conf, class_id in zip(results['boxes'], 
                                         results['confidences'], 
                                         results['class_ids']):
                x1, y1, x2, y2 = box.astype(int)
                label = self.class_names[class_id]
                color = colors[class_id]

                # Draw box
                rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                   fill=False,
                                   color=color,
                                   linewidth=2)
                plt.gca().add_patch(rect)

                # Add label
                label_text = f'{label} {conf:.2f}'
                plt.text(x1, y1-10, label_text,
                        color=color,
                        fontsize=8,
                        bbox=dict(facecolor='white', alpha=0.7))

            plt.title(f'Total Detections: {results["total_objects"]}\n'
                     f'Inference Time: {results["inference_time"]:.3f} seconds')
            plt.axis('off')
            plt.tight_layout()

            return plt.gcf()

        except Exception as e:
            print(f"Error during visualization: {e}")
            raise

    def print_detection_stats(self, image_path, conf_threshold=0.25):
        """
        Print detailed statistics about detections
        """
        try:
            results, _ = self.detect_objects(image_path, conf_threshold)

            print("\nDetection Statistics:")
            print("=" * 50)
            print(f"\nTotal Objects Detected: {results['total_objects']}")
            print(f"Inference Time: {results['inference_time']:.3f} seconds")

            print("\nDetections by Class:")
            print("-" * 50)
            print(f"{'Class':<20} {'Count':<10} {'Avg Confidence':<15}")
            print("-" * 50)

            for class_name, data in results['class_counts'].items():
                avg_conf = np.mean(data['confidences'])
                print(f"{class_name:<20} {data['count']:<10} {avg_conf:.3f}")

        except Exception as e:
            print(f"Error printing statistics: {e}")
            raise

def main():
    # Initialize detector with your quantized model
    model_path = "yolov8n_quantized.pt"  # Replace with your actual quantized model path
    detector = QuantizedDetector(model_path)

    # Path to test image
    image_path = "images/3892.jpg"  # Replace with your image path

    try:
        # Print detection statistics
        detector.print_detection_stats(image_path)

        # Visualize detections
        fig = detector.visualize_detections(image_path)
        plt.show()

    except Exception as e:
        print(f"Error in main: {e}")

if __name__ == "__main__":
    main()